<center>
    <h3>Arnaud Chéridi</h3>
    <p>Étudiant en Master 2 Mathématiques Appliquées</p>
    <p>Projet individuel : Modélisation du risque de crédit (cas IFRS 9)</p>
    <p>Objectif : développer un modèle de PD (score > 65%) + LGD + simulation</p>
</center>

<center><h1>Projet : Modélisation du risque de crédit – Cas type IFRS 9</h1></center>

<center><h2> 1. Préparation des données et des variables <h2><center>

## 0. Import des bibliothèques

In [1]:
import pandas as pd
import numpy as np

## 1. Chargements des données

Ces fichiers correspondent aux différentes sources utilisées pour modéliser le risque de crédit (client actuel, historique, crédits précédents, etc.).

In [2]:
data_path = 'data/'

In [3]:
app_train = pd.read_csv(f'{data_path}/application_train.csv')
app_test=pd.read_csv(f'{data_path}/application_test.csv')
bureau = pd.read_csv(f'{data_path}/bureau.csv')
bureau_balance = pd.read_csv(f'{data_path}/bureau_balance.csv')
pos_cash_balance = pd.read_csv(f'{data_path}/POS_CASH_balance.csv')
credit_card_balance = pd.read_csv(f'{data_path}/credit_card_balance.csv')
previous_application = pd.read_csv(f'{data_path}/previous_application.csv')
installments_payments = pd.read_csv(f'{data_path}/installments_payments.csv')

## 2. Aggrégation des données

### 2.1 Bureau

Aggrégation des données issues du bureau gérant les crédit externes.

In [4]:
# Récupération de la clé externe et interne
bureau_balance_group = bureau_balance.groupby('SK_ID_BUREAU').agg(
    month_min = ("MONTHS_BALANCE", "min"),
    month_max = ("MONTHS_BALANCE", "max"),
    status_last = ("STATUS", "last")
).reset_index()

In [5]:
bureau = bureau.merge(bureau_balance, on='SK_ID_BUREAU', how='left')

In [6]:
bureau["CREDIT_TYPE"].unique()

array(['Consumer credit', 'Credit card', 'Mortgage', 'Car loan',
       'Microloan', 'Loan for working capital replenishment',
       'Loan for business development', 'Real estate loan',
       'Unknown type of loan', 'Another type of loan',
       'Cash loan (non-earmarked)', 'Loan for the purchase of equipment',
       'Mobile operator loan', 'Interbank credit',
       'Loan for purchase of shares (margin lending)'], dtype=object)

In [7]:
# Création d'une liste de crédit avec le plus de risque
risky_types = ["Consumer credit", "Credit card", "Cash loan (non-earmarked)", "Microloan"]

On agrège les crédits passés de chaque client auprès d'autres établissements pour en extraire :
- des volumes (nb_credits, total_credit_sum),
- des risques (total_credit_debt, avg_debt_ratio, max_overdue),
- des comportements (prolongations, types de crédits),
- et une dynamique temporelle (ancienneté, activité).

Ces indicateurs permettent de mieux évaluer la situation financière globale du client.

In [8]:
bureau_agg = bureau.groupby('SK_ID_CURR').agg(
    nb_credits=("SK_ID_BUREAU", "count"),
    total_credit_sum=("AMT_CREDIT_SUM", "sum"),
    total_credit_debt=("AMT_CREDIT_SUM_DEBT", "sum"),
    avg_credit_amount=("AMT_CREDIT_SUM", "mean"),
    avg_debt_ratio=("AMT_CREDIT_SUM_DEBT", lambda debt: (debt / bureau.loc[debt.index, "AMT_CREDIT_SUM"]).replace([np.inf, -np.inf], np.nan).mean()),
    max_overdue=("AMT_CREDIT_SUM_OVERDUE", "max"),
    mean_days_credit=("DAYS_CREDIT", "mean"),
    min_days_credit=("DAYS_CREDIT", "min"),
    nb_prolongs=("CNT_CREDIT_PROLONG", "sum"),
    avg_annuity=("AMT_ANNUITY", "mean"),
    nb_credits_active=("CREDIT_ACTIVE", lambda x: (x == "Active").sum()),
    nb_credits_closed=("CREDIT_ACTIVE", lambda x: (x == "Closed").sum()),
	part_credits_risked=("CREDIT_TYPE", lambda x: x.isin(risky_types).mean()),
).reset_index()

### 2.2 Installments_payments

On regroupe les échéances de chaque crédit pour calculer :
- les montants dus/payés,
- les retards moyens et extrêmes,
- la fréquence des paiements manquants,
- ainsi que le taux de remboursement global.

Ces variables reflètent la régularité du client dans le remboursement de ses crédits passés.

In [9]:
inst_by_credit = installments_payments.groupby("SK_ID_PREV").agg(
    SK_ID_CURR=("SK_ID_CURR", "first"),
    total_due=("AMT_INSTALMENT", "sum"),
    total_paid=("AMT_PAYMENT", "sum"),
    nb_paiements=("SK_ID_PREV", "count"),
    retard_moyen=("DAYS_ENTRY_PAYMENT", lambda x: (x - installments_payments.loc[x.index, "DAYS_INSTALMENT"]).mean()),
    retards_proportion=("DAYS_ENTRY_PAYMENT", lambda x: (x > installments_payments.loc[x.index, "DAYS_INSTALMENT"]).mean()),
    max_retard=("DAYS_ENTRY_PAYMENT", lambda x: (x - installments_payments.loc[x.index, "DAYS_INSTALMENT"]).max()),
    paiements_manquants=("AMT_PAYMENT", lambda x: (x == 0).mean())
)

inst_by_credit["taux_remboursement"] = inst_by_credit["total_paid"] / inst_by_credit["total_due"]

In [10]:
# Identification du nombre de ligne ou la capital est déja remboursé
(inst_by_credit["total_due"]==0).sum()

np.int64(16)

In [11]:
# Vérification pas d'erreur dans le report du montant total dû
inst_by_credit["total_due"].min()

np.float64(0.0)

In [12]:
# Suppression des lignes dont le capital est déja remboursé
inst_by_credit = inst_by_credit[inst_by_credit["total_due"] > 0]

On passe du niveau crédit au niveau client en calculant des moyennes simples et pondérées (par le montant dû) :
- pour les montants remboursés,
- la ponctualité des paiements,
- la fréquence des retards ou défauts.

Cela permet de capturer la discipline de remboursement d’un client en tenant compte de l’importance financière de chaque crédit.

In [13]:
# On définit les poids : mensualité moyenne
inst_by_credit["mean_due"] = inst_by_credit["total_due"] / inst_by_credit["nb_paiements"]

# Agrégation pondérée au niveau client
inst_agg = inst_by_credit.groupby("SK_ID_CURR").agg(
    nb_credits=("SK_ID_CURR", "count"),
    montant_du_total=("total_due", "sum"),
    total_nb_paiements=("nb_paiements", "sum"),
    mean_due_moyen=("mean_due", "mean"),
    mean_due_pondere=("mean_due", lambda x: np.average(x, weights=inst_by_credit.loc[x.index, "total_due"])),
    montant_paye_total=("total_paid", "sum"),
    taux_remboursement_pondere=("taux_remboursement", lambda x: np.average(x, weights=inst_by_credit.loc[x.index, "total_due"])),
    retard_moyen_pondere=("retard_moyen", lambda x: np.average(x, weights=inst_by_credit.loc[x.index, "total_due"])),
    retards_proportion_pondere=("retards_proportion", lambda x: np.average(x, weights=inst_by_credit.loc[x.index, "total_due"])),
    max_retard=("max_retard", "max"),  # non pondéré, car on garde le max
    paiements_manquants_pondere=("paiements_manquants", lambda x: np.average(x, weights=inst_by_credit.loc[x.index, "total_due"]))
).reset_index()

### 2.3 Previous_application

On synthétise l’historique de demandes passées par client pour extraire :
- des volumes (`nb_demandes`, `nb_demandes_annulées`),
- des indicateurs de risque ou de profil (`taux_refus`, `rapport_demande_accord`),
- des comportements types (`type_credit_principal`, `délai_moyen_décision`).

Ces variables aident à caractériser la relation passée du client avec le crédit, et à détecter les profils à haut risque.

In [14]:
prev_agg = previous_application.groupby("SK_ID_CURR").agg(
    nb_demandes=("SK_ID_PREV", "count"),
    taux_refus=("NAME_CONTRACT_STATUS", lambda x: (x == "Refused").mean()),
    montant_moyen_demande=("AMT_APPLICATION", "mean"),
    montant_moyen_accordé=("AMT_CREDIT", "mean"),
    rapport_demande_accord=("AMT_CREDIT", lambda x: (x / previous_application.loc[x.index, "AMT_APPLICATION"].replace(0, np.nan)).mean()),
    type_credit_principal=("NAME_CONTRACT_TYPE", lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
    nb_demandes_annulees=("NAME_CONTRACT_STATUS", lambda x: (x == "Canceled").sum()),
    delai_moyen_decision=("DAYS_DECISION", "mean"),
).reset_index()

### 2.4 Credit_card_balance

On regroupe les informations d’utilisation de chaque carte de crédit pour évaluer :
- l’intensité d’utilisation (solde, paiements, limite, taux d’utilisation),
- le suivi dans le temps (durée d’observation, nb de mois),
- la régularité des remboursements (`dpd`, retards).

Ces indicateurs aident à détecter les comportements à risque liés à la gestion des crédits renouvelables.

In [15]:
credit_card_by_card = credit_card_balance.groupby("SK_ID_PREV").agg(
    SK_ID_CURR=("SK_ID_CURR", "first"),
    nb_mois_activite=("MONTHS_BALANCE", "count"),
    duree_observation=("MONTHS_BALANCE", lambda x: x.max() -  x.min() + 1),
    solde_moyen=("AMT_BALANCE", "mean"),
    limite_moyenne=("AMT_CREDIT_LIMIT_ACTUAL", "mean"),
    taux_utilisation_moyen=("AMT_BALANCE", lambda x: (
        (x / credit_card_balance.loc[x.index, "AMT_CREDIT_LIMIT_ACTUAL"].replace(0, np.nan)).mean()
    )),
    paiement_moyen=("AMT_PAYMENT_TOTAL_CURRENT", "mean"),
    dpd_moyen=("SK_DPD", "mean"),
    dpd_max=("SK_DPD", "max"),
    depassement_freq=("SK_DPD", lambda x: (x > 0).mean())
).reset_index()

In [17]:
# On s’assure que pour chaque carte, le nombre de mois d’activité correspond bien à la durée observée
(credit_card_by_card["nb_mois_activite"] == credit_card_by_card["duree_observation"]).sum()/len(credit_card_by_card)

np.float64(0.9998657808200792)

In [18]:
# Vérification que la limite
credit_card_by_card["limite_moyenne"].max()

np.float64(1350000.0)

In [19]:
# Vérification que la limite
credit_card_by_card["limite_moyenne"].min()

np.float64(0.0)

In [20]:
# Mise en place d'une pondération
credit_card_by_card["poids"] = credit_card_by_card["nb_mois_activite"] * credit_card_by_card["limite_moyenne"]

In [21]:
(credit_card_by_card["poids"]==0).sum()

np.int64(1173)

In [22]:
credit_card_by_card["poids"].min()

np.float64(0.0)

In [23]:
# Suppression des poids nuls
credit_card_by_card = credit_card_by_card[credit_card_by_card["poids"] > 0]

On agrège les informations par client en tenant compte du "poids" de chaque carte (durée × limite) :
- exposition totale et nombre de cartes,
- moyenne pondérée des soldes, limites, taux d'utilisation et retards,
- mesure de risque maximale (`dpd_max`).

Cette pondération permet de mieux refléter l’impact des cartes les plus actives ou importantes.

In [24]:
credit_card_agg = credit_card_by_card.groupby("SK_ID_CURR").agg(
    nb_cartes=("SK_ID_PREV", "count"),
    exposition_totale=("poids", "sum"),
    total_duree_cartes=("duree_observation", "sum"),

    solde_pondere=("solde_moyen", lambda x: np.average(
        x, weights=credit_card_by_card.loc[x.index, "poids"]
    )),

    limite_ponderee=("limite_moyenne", lambda x: np.average(
        x, weights=credit_card_by_card.loc[x.index, "poids"]
    )),

    taux_utilisation_pondere=("taux_utilisation_moyen", lambda x: np.average(
        x, weights=credit_card_by_card.loc[x.index, "poids"]
    )),

    dpd_moyen_pondere=("dpd_moyen", lambda x: np.average(
        x, weights=credit_card_by_card.loc[x.index, "poids"]
    )),

    depassement_freq_ponderee=("depassement_freq", lambda x: np.average(
        x, weights=credit_card_by_card.loc[x.index, "poids"]
    )),

    dpd_max=("dpd_max", "max")
).reset_index()

### 2.5 Posh_cash_balance

On calcule, en pondérant par durée × montant dû :
- l’exposition totale aux crédits POS,
- les retards moyens et extrêmes (`dpd`, `dpd_defaut`).

Ces indicateurs permettent d’évaluer la gestion du crédit renouvelable, souvent associé à un risque accru de défaut.

In [25]:
pos_cash_by_credit = pos_cash_balance.groupby("SK_ID_PREV").agg(
    SK_ID_CURR=("SK_ID_CURR", "first"),
    duree_observation=("MONTHS_BALANCE", "count"),
    nb_echeances_totales=("CNT_INSTALMENT", "max"),
    dpd_moyen=("SK_DPD", "mean"),
    dpd_defaut_moyen=("SK_DPD_DEF", "mean"),
    dpd_max=("SK_DPD", "max"),
    dpd_defaut_max=("SK_DPD_DEF", "max")
).reset_index()

In [26]:
(pos_cash_by_credit["nb_echeances_totales"].isnull()).sum()

np.int64(890)

In [27]:
(pos_cash_by_credit["duree_observation"].isnull()).sum()

np.int64(0)

In [28]:
# Mise en place d'une pondération
pos_cash_by_credit["poids"] = (
    pos_cash_by_credit["nb_echeances_totales"]*
    pos_cash_by_credit["duree_observation"]
)

In [29]:
# Suppression des lignes dont la pondération est nulle
pos_cash_by_credit = pos_cash_by_credit[pos_cash_by_credit["nb_echeances_totales"].isnull() == False]

On regroupe les crédits renouvelables POS au niveau client, en pondérant les indicateurs par l’exposition (durée × montant dû) :
- nombre total de crédits POS,
- exposition cumulée,
- retards moyens et maximaux (`dpd`, `dpd_defaut`).

Ces variables permettent de mesurer la pression financière liée à ce type de crédit, souvent plus risqué.

In [30]:
pos_cash_agg = pos_cash_by_credit.groupby("SK_ID_CURR").agg(
    nb_credits_pos=("SK_ID_PREV", "count"),
    exposition_pos=("poids", "sum"),
    dpd_moyen_pondere=("dpd_moyen", lambda x: np.average(x, weights=pos_cash_by_credit.loc[x.index, "poids"])),
    dpd_max=("dpd_max", "max"),
    dpd_defaut_moyen_pondere=("dpd_defaut_moyen", lambda x: np.average(x, weights=pos_cash_by_credit.loc[x.index, "poids"])),
    dpd_defaut_max=("dpd_defaut_max", "max")
).reset_index()

### 2.6 Création d'une variable de LGD simulée

On approxime la Loss Given Default (LGD) pour chaque crédit précédent en calculant :

- le taux de perte simulé `LGD_sim = 1 - (payment / instalment)` (censuré entre 0 et 1),
- une moyenne pondérée par le montant dû (pour refléter l’importance de chaque échéance),
- puis on fusionne ces estimations au niveau du crédit (`SK_ID_PREV`) avec les métadonnées (`AMT_CREDIT`, type de contrat...).

On conserve uniquement les crédits pour lesquels on peut estimer une LGD valide (non nulle, non manquante).

In [31]:
info_credit = previous_application[['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CONTRACT_TYPE', 'AMT_CREDIT']]
inst_lgd = installments_payments[["SK_ID_PREV", "AMT_PAYMENT", "AMT_INSTALMENT"]].copy()
inst_lgd["LGD_sim"] = 1 - (inst_lgd["AMT_PAYMENT"] / inst_lgd["AMT_INSTALMENT"]).clip(lower=0, upper=1)
inst_lgd = inst_lgd[inst_lgd["AMT_INSTALMENT"] > 0]
lgd_sim_agg = inst_lgd.groupby("SK_ID_PREV").agg(
    lgd_sim_moyen_pondere=("LGD_sim", lambda x: np.average(x, weights=inst_lgd.loc[x.index, "AMT_INSTALMENT"])),
).reset_index()
info_credit = info_credit.merge(lgd_sim_agg, on="SK_ID_PREV", how="left")
info_credit = info_credit[info_credit["lgd_sim_moyen_pondere"].notna()]

On agrège les estimations de LGD au niveau du client (`SK_ID_CURR`),
en créant une colonne par type de contrat (`Cash loans`, `Consumer loans`, etc.).

Cela permet de capturer le risque de perte moyen pour chaque type de crédit contracté par le client.

In [32]:
lgd_pivot = info_credit.pivot_table(
    index="SK_ID_CURR",
    columns="NAME_CONTRACT_TYPE",
    values="lgd_sim_moyen_pondere"
)

### 2.7 Application

In [33]:
# Mise en place d'un marqueur
app_train["dataset"] = "train"
app_test["dataset"] = "test"

# Regrouppement des fichiers train et test
full_app = pd.concat([app_train, app_test], axis=0)

On dérive des variables continues clés :
- âge et ancienneté professionnelle (en années),
- ratios d’endettement relatifs au revenu (`annuity/income`, `credit/income`),
- ratio entre le crédit accordé et la valeur du bien (`credit/goods`).

Ces indicateurs permettent de quantifier la soutenabilité financière et la stabilité du client.

In [34]:
full_app["age"] = -full_app["DAYS_BIRTH"] / 365
full_app["anciennete_emploi"] = -full_app["DAYS_EMPLOYED"] / 365
full_app["ratio_annuity_income"] = full_app["AMT_ANNUITY"] / full_app["AMT_INCOME_TOTAL"]
full_app["ratio_credit_income"] = full_app["AMT_CREDIT"] / full_app["AMT_INCOME_TOTAL"]
full_app["ratio_credit_goods"] = full_app["AMT_CREDIT"] / full_app["AMT_GOODS_PRICE"]

On supprime :
- des colonnes documentaires (`FLAG_DOCUMENT`) peu discriminantes,
- des variables de redondance géographique ou temporelle,
- des détails sur le logement ou le bâtiment souvent manquants ou non exploitables (`_AVG`, `_MODE`, `WALLSMATERIAL`, etc.).

Cette étape permet de réduire la dimensionnalité et d’éviter le sur-apprentissage sur des variables peu robustes.

In [35]:
cols_to_drop = [
    # Flags documentaires
    *[f"FLAG_DOCUMENT_{i}" for i in range(2, 22)],

    # Infos bruitées / redondantes
    "DAYS_ID_PUBLISH", "DAYS_REGISTRATION", "FLAG_EMP_PHONE",
    "WEEKDAY_APPR_PROCESS_START", "HOUR_APPR_PROCESS_START",
    "LIVE_CITY_NOT_WORK_CITY", "REG_CITY_NOT_WORK_CITY",
    "LIVE_REGION_NOT_WORK_REGION", "REG_REGION_NOT_WORK_REGION",
    "REG_CITY_NOT_LIVE_CITY", "REG_REGION_NOT_LIVE_REGION",

    # Infos bâtiment / zone
    *[col for col in full_app.columns if any(substr in col for substr in
      ["_AVG", "_MODE", "_MEDI", "WALLSMATERIAL", "FONDKAPREMONT",
       "HOUSETYPE", "TOTALAREA", "EMERGENCYSTATE"])]
]

full_app = full_app.drop(cols_to_drop, axis=1)

## 3. Regroupement des informations

On renomme les colonnes agrégées avec un préfixe explicite (`bureau_`, `install_`, etc.) tout en conservant l’identifiant client.
Puis, on fusionne toutes les données secondaires (crédit, POS, cartes, etc.) avec `full_app`.

In [36]:
def add_prefix_keep_id(df, prefix, id_col="SK_ID_CURR"):
    df_temp = df.copy()
    id_series = df_temp[id_col]
    df_prefixed = df_temp.drop(columns=id_col).add_prefix(f"{prefix}_")
    df_prefixed[id_col] = id_series
    return df_prefixed

# Application aux différents fichiers agrégés
bureau_agg      = add_prefix_keep_id(bureau_agg, "bureau")
inst_agg        = add_prefix_keep_id(inst_agg, "install")
prev_agg        = add_prefix_keep_id(prev_agg, "prev")
credit_card_agg = add_prefix_keep_id(credit_card_agg, "cc")
pos_cash_agg    = add_prefix_keep_id(pos_cash_agg, "pos")

full_df = full_app.copy()

full_df = full_df.merge(bureau_agg, on="SK_ID_CURR", how="left")
full_df = full_df.merge(inst_agg, on="SK_ID_CURR", how="left")
full_df = full_df.merge(prev_agg, on="SK_ID_CURR", how="left")
full_df = full_df.merge(credit_card_agg, on="SK_ID_CURR", how="left")
full_df = full_df.merge(pos_cash_agg, on="SK_ID_CURR", how="left")

On attribue à chaque ligne de `full_app` la LGD moyenne simulée correspondant à :
- son identifiant client (`SK_ID_CURR`),
- et au type de crédit (`NAME_CONTRACT_TYPE`),

en s’appuyant sur le tableau croisé `lgd_pivot` calculé précédemment.

Cela permet d’enrichir chaque demande actuelle avec une estimation du risque de perte basée sur l’historique réel du client (ou de clients similaires).

In [37]:
full_df["LGD_sim"] = list(map(
    lambda row:
        lgd_pivot.at[row[0], row[1]]                     # récupère la valeur LGD pour (client, type contrat)
        if row[0] in lgd_pivot.index and row[1] in lgd_pivot.columns  # si la combinaison existe
        else np.nan,                                     # sinon on met NaN
    zip(full_app["SK_ID_CURR"], full_app["NAME_CONTRACT_TYPE"])  # on crée un tuple (ID, type contrat) par ligne
))

## 4. Traitement des valeurs manquantes

On calcule la proportion de valeurs manquantes pour chaque variable.
Cela permet d’identifier les colonnes à traiter par imputation ou à supprimer avant modélisation.

In [38]:
missing_ratio = full_df.isnull().mean().sort_values(ascending=False)
missing_ratio[missing_ratio > 0.0]

cc_solde_pondere                0.712439
cc_nb_cartes                    0.712439
cc_exposition_totale            0.712439
cc_dpd_max                      0.712439
cc_depassement_freq_ponderee    0.712439
                                  ...   
AMT_GOODS_PRICE                 0.000780
ratio_annuity_income            0.000101
AMT_ANNUITY                     0.000101
CNT_FAM_MEMBERS                 0.000006
DAYS_LAST_PHONE_CHANGE          0.000003
Length: 71, dtype: float64

1. Suppression de certaines variables inutiles ou redondantes (`bureau_avg_annuity`).
2. Pour les colonnes issues des cartes de crédit (`cc_`) et `LG_sim`, on :
   - crée un indicateur binaire de valeur manquante (`col_NA`) pour informer les modèles ;
   - impute les valeurs par la médiane.
3. Imputation ciblée sur certaines variables importantes (`OWN_CAR_AGE`, `EXT_SOURCE`, etc.) :
   - par la médiane pour les variables numériques,
   - par "Unknown" pour les catégories comme `OCCUPATION_TYPE`.

Enfin, on applique une imputation automatique à toutes les colonnes restantes :
   - **numériques** → médiane,
   - **catégorielles** → `"Unknown"`.

Ce traitement permet d'éviter les biais dus aux valeurs manquantes tout en informant le modèle sur leur présence.

In [39]:
# Suppression de la colonne bureau_avg_annuity
full_df = full_df.drop(columns=["bureau_avg_annuity"])

In [40]:
# Étape 1 : identifier les colonnes commençant par "cc_"
cc_cols = [col for col in full_df.columns if col.startswith("cc_") or col.startswith("LGD")]

# Étape 2 : pour chaque colonne cc_
for col in cc_cols:
    # Crée une variable binaire indiquant si la valeur était manquante
    full_df[f"{col}_NA"] = full_df[col].isnull().astype(int)

    # Impute les NaN par la médiane
    median_val = full_df[col].median()
    full_df[col] = full_df[col].fillna(median_val)

In [41]:
full_df["OWN_CAR_AGE_nan"] = full_df["OWN_CAR_AGE"].isna().astype(int)
full_df["OWN_CAR_AGE"] = full_df["OWN_CAR_AGE"].fillna(full_df["OWN_CAR_AGE"].median())

In [42]:
full_df["EXT_SOURCE__nan"] = full_df["EXT_SOURCE_1"].isna().astype(int)
full_df["EXT_SOURCE_1"] = full_df["EXT_SOURCE_1"].fillna(full_df["EXT_SOURCE_1"].median())

In [43]:
full_df["OCCUPATION_TYPE"] = full_df["OCCUPATION_TYPE"].fillna("Unknown")

In [44]:
full_df["EXT_SOURCE_3_nan"] = full_df["EXT_SOURCE_3"].isna().astype(int)
full_df["EXT_SOURCE_3"] = full_df["EXT_SOURCE_3"].fillna(full_df["EXT_SOURCE_3"].median())

In [45]:
missing_cols = full_df.columns[full_df.isna().any()]

for col in missing_cols:
    if full_df[col].dtype in ['float64', 'int64']:
        full_df[col] = full_df[col].fillna(full_df[col].median())
    else:
        full_df[col] = full_df[col].fillna("Unknown")

## 5. Finalisation des fichiers train et test prêts à la modélisation

In [46]:
train_final = full_df[full_df["dataset"] == "train"].drop(columns="dataset").copy()
test_final = full_df[full_df["dataset"] == "test"].drop(columns=["dataset", "TARGET"]).copy()

In [47]:
train_final.to_csv("data/train_final.csv", index=False)
test_final.to_csv("data/test_final.csv", index=False)